# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

# If necessary, change directory to the [github page]/markdown_generator page
# os.chdir('small_projects//knevel_lab_web_page//knevel-lab.github.io//markdown_generator')

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [20]:
!type talks.tsv
# with open('talks.tsv', 'r') as fh:
#     print(fh.read())

title	type	url_slug	venue	date	location	talk_url	description
EULAR 2024 Rheumatic	Poster	2024_FZ_RHEUMATIC	EULAR 2024	13-06-2024	Vienna, Austria	https://scientific.sparx-ip.net/archiveeular/?c=a&item=2024POS0750	Who participates in an online symptom checker? Characteristics, attrition and representativeness in the Rheumatic? study
EULAR 2024 ML Diagnoses	Poster	2024_TDM_EULAR_ML_DIAG	EULAR 2024	13-06-2024	Vienna, Austria	https://scientific.sparx-ip.net/archiveeular/?c=a&item=2024POS0587	Employing machine learning to predict RA diagnosis from referral letters by the general practitioner
EULAR 2024 RA subsets	Poster	2024_TDM_EULAR_RA_SUBSETS	EULAR 2024	13-06-2024	Vienna, Austria	https://scientific.sparx-ip.net/archiveeular/?c=a&item=2024POS0589	EHR defined baseline RA-subsets characterised by localised joint involvement differentially associate with genetic risk and clinical outcomes beyond baseline
EULAR 2024 EUMENET	Presentation	2024_RK_EUMENET	EULAR 2024	12-06-2024	Vienna, Austria	htt

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [12]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,EULAR 2024 Rheumatic,Poster,2024_FZ_RHEUMATIC,EULAR 2024,13-06-2024,"Vienna, Austria",https://scientific.sparx-ip.net/archiveeular/?...,Who participates in an online symptom checker?...
1,EULAR 2024 ML Diagnoses,Poster,2024_TDM_EULAR_ML_DIAG,EULAR 2024,13-06-2024,"Vienna, Austria",https://scientific.sparx-ip.net/archiveeular/?...,Employing machine learning to predict RA diagn...
2,EULAR 2024 RA subsets,Poster,2024_TDM_EULAR_RA_SUBSETS,EULAR 2024,13-06-2024,"Vienna, Austria",https://scientific.sparx-ip.net/archiveeular/?...,EHR defined baseline RA-subsets characterised ...
3,EULAR 2024 EUMENET,Presentation,2024_RK_EUMENET,EULAR 2024,12-06-2024,"Vienna, Austria",https://apps-congress.eular.org/eular2024/en-G...,How to establish a successful European researc...
4,EULAR 2024 Patient trajectories,Oral Presentation,2024_NS_EULAR,EULAR 2024,12-06-2024,"Vienna, Austria",https://scientific.sparx-ip.net/archiveeular/?...,Dissecting early RA patïent trajectories throu...
5,EULAR 2022 EHR Clustering,Poster,2022_TDM_EULAR,EULAR 2022,2022-06-01,"Copenhagen, Denmark",http://scientific.sparx-ip.net/archiveeular/?c...,Poster presentation of the work performed by T...
6,EULAR 2019 ICD Clustering,Abstract,2019_MPM_EULAR,EULAR 2019,2019-06-12,"Madrid, Spain",https://scientific.sparx-ip.net/archiveeular/?...,Accepted abstract of the work performed by MP ...
7,EULAR 2021 SSc Risk Model,Poster,2021_MPM_EULAR_1,EULAR 2021,2021-06-02,Online,http://scientific.sparx-ip.net/archiveeular/?c...,Poster presentation of the work performed by M...
8,EULAR 2019 Text Mining Dx,Poster Tour,2019_TDM_EULAR_1,EULAR 2019,2019-06-12,"Madrid, Spain",http://scientific.sparx-ip.net/archiveeular/?c...,Poster tour presentation of the work performed...
9,EULAR 2019 Text Mining Rx,Poster Tour,2019_TDM_EULAR_2,EULAR 2019,2019-06-12,"Madrid, Spain",http://scientific.sparx-ip.net/archiveeular/?c...,Poster tour presentation of the work performed...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [13]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [18]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("..//_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [19]:
!dir ..\_talks

 Volume in drive D is datasafe$
 Volume Serial Number is 009A-9A03

 Directory of D:\small_projects\knevel_lab_web_page\knevel-lab.github.io\_talks

17-06-2024  13:26    <DIR>          .
17-06-2024  10:59    <DIR>          ..
17-06-2024  13:17               411 12-06-2024-2024_FZ_RHEUMATIC.md
17-06-2024  13:35               476 12-06-2024-2024_NS_EULAR.md
17-06-2024  13:35               356 12-06-2024-2024_RK_EUMENET.md
17-06-2024  13:17               399 12-06-2024-2024_TDM_EULAR_ML_DIAG.md
17-06-2024  13:17               457 12-06-2024-2024_TDM_EULAR_RA_SUBSETS.md
17-06-2024  13:35               411 13-06-2024-2024_FZ_RHEUMATIC.md
17-06-2024  13:35               399 13-06-2024-2024_TDM_EULAR_ML_DIAG.md
17-06-2024  13:35               457 13-06-2024-2024_TDM_EULAR_RA_SUBSETS.md
17-06-2024  13:35               272 2018-09-20-2018_RK_DGRh.md
17-06-2024  13:35               314 2019-04-02-2019_MPM_BioSB.md
17-06-2024  13:35               391 2019-06-12-2019_MPM_EULAR.md
17-06-2024  13:35

In [7]:
!type ../_talks/2013-03-01-tutorial-1.md

---
title: "Tutorial 1 on Relevant Topic in Your Field"
collection: talks
type: "Tutorial"
permalink: /talks/2013-03-01-tutorial-1
venue: "UC-Berkeley Institute for Testing Science"
date: 2013-03-01
location: "Berkeley CA, USA"
---

[More information here](http://exampleurl.com)

This is a description of your tutorial, note the different field in type. This is a markdown files that can be all markdown-ified like any other post. Yay markdown!
